In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# Aux Function for filtering data

def filter_by_points(df, frequency='D', num_points=1440, return_dictionary=False):
    
    df_dropped = df.dropna()
    grouper = df_dropped.groupby(pd.Grouper(freq=frequency))
    
    output = 0
    if return_dictionary:
        new_dict = {}
        for i in grouper:
            if (len(i[1]) != num_points):
                pass
            else:
                new_dict[i[0]] = pd.DataFrame(i[1])
        output = new_dict
    else:
        new_df = pd.DataFrame({})
        for i in grouper:
            if (len(i[1]) != num_points):
                pass
            else:
                new_df = new_df.append(pd.DataFrame(i[1]))
        output = new_df
            
    return output

In [4]:
# Load the toy data

# Agent 01
agent01_consumption = pd.read_csv('data/consumption_agent_01.csv')
agent01_consumption.index = pd.to_datetime(agent01_consumption['timestamp'])
agent01_consumption.drop('timestamp', axis=1, inplace=True)

agent01_generation = pd.read_csv('data/generation_agent_01.csv')
agent01_generation.index = pd.to_datetime(agent01_generation['timestamp'])
agent01_generation.drop('timestamp', axis=1, inplace=True)

# Merge agent 01
agent01 = agent01_generation.join(agent01_consumption, how='inner', rsuffix='_cons')
agent01.dropna(inplace=True)
agent01 = filter_by_points(agent01, frequency='D', num_points=1440/5)

# Agent 05
agent05_consumption = pd.read_csv('data/consumption_agent_05.csv')
agent05_consumption.index = pd.to_datetime(agent05_consumption['timestamp'])
agent05_consumption.drop('timestamp', axis=1, inplace=True)

agent05_generation = pd.read_csv('data/generation_agent_05.csv')
agent05_generation.index = pd.to_datetime(agent05_generation['timestamp'])
agent05_generation.drop('timestamp', axis=1, inplace=True)

agent05 = agent05_generation.join(agent05_consumption, how='inner', rsuffix='_cons')
agent05.dropna(inplace=True)
agent05 = filter_by_points(agent05, frequency='D', num_points=1440/5)

In [18]:
# Define the main class to be extended

class PowerObject(object):
    
    def __init__(self, identifier, 
                 production_forecasts=None, consumption_forecasts=None, 
                 production_gt=None, consumption_gt=None, powerCost=None):
        self.identifier = identifier
        
        # Forecast setting
        self.production_forecasts = production_forecasts
        self.consumption_forecasts = consumption_forecasts
        
        # Ground truth setting
        self.production_gt = production_gt
        self.consumption_gt = consumption_gt
        
        # Power states
        self.canSell = False
        self.excessEnergy = 0
        
        self.isBuying = False
        self.toImport = 0
        
        # Current House balance
        self.balance = 0
        self.new_balance = 0 # new value after market
        
        # Energy Pricing
        self.powerCost = powerCost
        
        # Logs
        self.logs = {
            'step': [],
            'prod_forecast': [],
            'cons_forecast': [],
            'balance': [],
            'new_balance': [],
            'canSell': [],
            'excessEnergy': [],
            'isBuying': [],
            'toImport': [],
            'action': [],
            'boughtFrom': [],
            'soldTo': []}
        
    def checkSales(self, step):
        if self.production_forecasts[step] >= self.consumption_forecasts[step]:
            self.canSell = True
        else:
            self.canSell = False
        return
    
    def calculateExcess(self, step):
        self.excessEnergy = self.production_forecasts[step] - self.consumption_forecasts[step] 
        return
        
    def doDecision(self):
        return
    
    def runMetrics(self):
        return
    
    def _step(self, step):
        # Calculate how much energy we need first
        self.balance = self.consumption_forecasts[step] - self.production_forecasts[step]
        
        # Check if we have extra - positive values, in order to sell
        self.checkSales(step)
        if self.canSell:
            self.calculateExcess(step)
        else:
            self.excessEnergy = -1
            
        # Check we are buying energy and how much
        self.toImport = np.max(self.balance, 0)
        if self.toImport > 0:
            self.isBuying = True
            
        self.new_balance = self.balance
        self.updateLogs(step)
        return
    
    def buyEnergy(self, step, quantity, seller_id, seller_price):
        if self.toImport == 0:
            return
        
        to_buy = min(self.toImport, quantity)
        self.toImport -= to_buy
        self.new_balance = self.balance - to_buy
        
        # Update logs
        self.updateLogs(step, action='Bought Energy', seller=seller_id)
        
        if (self.new_balance == 0) | (self.toImport == 0):
            self.isBuying = False
        
        print('{} bought {}W of required energy from {} for {:.02f}€ - still needs {}W'.format(self.identifier,
                                                                                       to_buy,
                                                                                       seller_id,
                                                                                       to_buy*seller_price,
                                                                                       self.toImport))
        return to_buy
    
    def sellEnergy(self, step, quantity, buyer_id):
        if self.canSell == False:
            return
        
        to_sell = min(quantity, self.excessEnergy)
        self.excessEnergy -= to_sell
        
        self.new_balance = self.balance + to_sell
        
        # Update the logs
        self.updateLogs(step, action='Sold Energy', buyer=buyer_id)
        
        if (self.new_balance == 0) | (self.excessEnergy == 0):
            self.canSell = False
        
        print('{} sold {}W of excessive energy to {} for {:.02f}€- has {}W remaining'.format(self.identifier, 
                                                                                     to_sell,
                                                                                     buyer_id,
                                                                                     self.powerCost*to_sell,
                                                                                     self.excessEnergy))
        return to_sell
    
    def updateLogs(self, step, action='None', buyer='None', seller='None'):
        self.logs['step'].append(step)
        self.logs['prod_forecast'].append(self.production_forecasts[step])
        self.logs['cons_forecast'].append(self.consumption_forecasts[step])
        self.logs['balance'].append(self.balance)
        self.logs['new_balance'].append(self.new_balance)
        self.logs['canSell'].append(self.canSell)
        self.logs['excessEnergy'].append(self.excessEnergy)
        self.logs['isBuying'].append(self.isBuying)
        self.logs['toImport'].append(self.toImport)
        self.logs['action'].append(action)
        self.logs['boughtFrom'].append(seller)
        self.logs['soldTo'].append(buyer)
        
        return

In [19]:
# Aggregator class for a Community

class PowerAggregator(object):
    
    def __init__(self, objs, max_iterations=500):
        self.controlling = objs
        
        # Number of maximum iterations to run
        self.max_iterations = max_iterations
        
        # Step counter for iterations
        self.step_counter = 0
        
        # Sellers and Buyers
        self.sellers = None
        self.buyers = None
        
    def getSellers(self):
        self.sellers = []
        for i in np.arange(len(self.controlling)):
            if self.controlling[i].canSell:
                self.sellers.append(self.controlling[i])
        return
    
    def getBuyers(self):
        self.buyers = []
        for i in np.arange(len(self.controlling)):
            if self.controlling[i].isBuying:
                self.buyers.append(self.controlling[i])
        return
        
    def _step(self, step):
        # Do the initial iteration for all the controlled objects
        for obj in np.arange(len(self.controlling)):
            self.controlling[obj]._step(step)
            
        self.getSellers()
        self.getBuyers()
        return
    
    def energyTrading(self, step, buyer, sellers):
        # First: Check energy buyers
        # Second: Check the cheapest sellers (powerCost values)
        # Third: Update values move to next controlled object
        
        best_seller = self.cheapest_seller()
        
        sold_amount = best_seller.sellEnergy(step, buyer.toImport, buyer.identifier)
        bought_amount = buyer.buyEnergy(step, sold_amount, 
                                        best_seller.identifier, 
                                        best_seller.powerCost)
        
        self.getSellers()
        self.getBuyers()
        return
        
        
    def cheapest_seller(self):
        seller_costs = []
        for i in np.arange(len(self.sellers)):
            seller_costs.append(self.sellers[i].powerCost)
            
        return self.sellers[np.argmin(seller_costs)]
    
    def iterateBuyers(self, step):
        self.energyTrading(step, self.buyers[0], self.sellers)
    
    def iterate(self):
        for i in np.arange(self.max_iterations):
            print('ITERATION {}'.format(i))
            self._step(i)
            
            while len(self.buyers) > 0:
                # do stuff
                self.iterateBuyers(i)
            
        return

In [21]:
# Class for service provider

class Provider(PowerObject):
    
    def __init__(self, 
                 identifier=None,
                 production_forecasts=None,
                 consumption_forecasts=None,
                 production_gt=None,
                 consumption_gt=None,
                 powerCost=None):
    
        super().__init__(identifier=identifier, 
                         production_forecasts=production_forecasts,
                         consumption_forecasts=consumption_forecasts,
                         production_gt=production_gt,
                         consumption_gt=consumption_gt,
                         powerCost=powerCost)
        
        self.canSell=True
        self.isBuying=False

In [22]:
# House class

class Household(PowerObject):
    
    def __init__(self, 
                 identifier=None,
                 production_forecasts=None,
                 consumption_forecasts=None,
                 production_gt=None,
                 consumption_gt=None,
                 powerCost=None):
        super().__init__(identifier=identifier, 
                         production_forecasts=production_forecasts,
                         consumption_forecasts=consumption_forecasts,
                         production_gt=production_gt,
                         consumption_gt=consumption_gt,
                         powerCost=powerCost)

In [23]:
house01 = Household(identifier='house01', 
                    production_forecasts=np.round(agent01['yhat'], 2), 
                    consumption_forecasts=np.round(agent01['yhat_cons'], 2), 
                    production_gt=np.round(agent01['y'], 2), 
                    consumption_gt=np.round(agent01['y_cons'], 2),
                    powerCost=0.05)

house05 = Household(identifier='house05',
                    production_forecasts=np.round(agent05['yhat'], 2),
                    consumption_forecasts=np.round(agent05['yhat_cons'], 2),
                    production_gt=np.round(agent05['y'], 2),
                    consumption_gt=np.round(agent05['y_cons'], 2),
                    powerCost=0.07)

provider01 = Provider(identifier='provider01',
                      production_forecasts=[9999 for i in np.arange(agent01.shape[0])],
                      consumption_forecasts=[0 for i in np.arange(agent01.shape[0])],
                      production_gt=[9999 for i in np.arange(agent01.shape[0])],
                      consumption_gt=[0 for i in np.arange(agent01.shape[0])],
                      powerCost=0.20)

aggregator01 = PowerAggregator(objs=[provider01, house01, house05], max_iterations=int(1440/5))
aggregator01.iterate()

ITERATION 0
provider01 sold 610.35W of excessive energy to house01 for 122.07€- has 9388.65W remaining
house01 bought 610.35W of required energy from provider01 for 122.07€ - still needs 0.0W
provider01 sold 81.62W of excessive energy to house05 for 16.32€- has 9307.029999999999W remaining
house05 bought 81.62W of required energy from provider01 for 16.32€ - still needs 0.0W
ITERATION 1
provider01 sold 610.35W of excessive energy to house01 for 122.07€- has 9388.65W remaining
house01 bought 610.35W of required energy from provider01 for 122.07€ - still needs 0.0W
provider01 sold 76.62W of excessive energy to house05 for 15.32€- has 9312.029999999999W remaining
house05 bought 76.62W of required energy from provider01 for 15.32€ - still needs 0.0W
ITERATION 2
provider01 sold 615.85W of excessive energy to house01 for 123.17€- has 9383.15W remaining
house01 bought 615.85W of required energy from provider01 for 123.17€ - still needs 0.0W
provider01 sold 80.96000000000001W of excessive ener

In [30]:
house02 = Household(identifier='house02', 
                    production_forecasts=np.round(agent01['yhat']*1.3, 2), 
                    consumption_forecasts=np.round(agent01['yhat_cons']*0.8, 2), 
                    production_gt=np.round(agent01['y']*1.3, 2), 
                    consumption_gt=np.round(agent01['y_cons']*0.8, 2),
                    powerCost=0.02)

house03 = Household(identifier='house03',
                    production_forecasts=np.round(agent05['yhat']*1.1, 2),
                    consumption_forecasts=np.round(agent05['yhat_cons']*1.2, 2),
                    production_gt=np.round(agent05['y']*1.1, 2),
                    consumption_gt=np.round(agent05['y_cons']*1.2, 2),
                    powerCost=0.07)

provider02 = Provider(identifier='provider01',
                      production_forecasts=[9999 for i in np.arange(agent01.shape[0])],
                      consumption_forecasts=[0 for i in np.arange(agent01.shape[0])],
                      production_gt=[9999 for i in np.arange(agent01.shape[0])],
                      consumption_gt=[0 for i in np.arange(agent01.shape[0])],
                      powerCost=0.20)

aggregator02 = PowerAggregator(objs=[provider02, house02, house03], max_iterations=int(1440/5))
aggregator02.iterate()

ITERATION 0
provider01 sold 488.28W of excessive energy to house02 for 97.66€- has 9510.72W remaining
house02 bought 488.28W of required energy from provider01 for 97.66€ - still needs 0.0W
provider01 sold 97.92W of excessive energy to house03 for 19.58€- has 9412.8W remaining
house03 bought 97.92W of required energy from provider01 for 19.58€ - still needs 0.0W
ITERATION 1
provider01 sold 488.28W of excessive energy to house02 for 97.66€- has 9510.72W remaining
house02 bought 488.28W of required energy from provider01 for 97.66€ - still needs 0.0W
provider01 sold 91.92W of excessive energy to house03 for 18.38€- has 9418.8W remaining
house03 bought 91.92W of required energy from provider01 for 18.38€ - still needs 0.0W
ITERATION 2
provider01 sold 492.68W of excessive energy to house02 for 98.54€- has 9506.32W remaining
house02 bought 492.68W of required energy from provider01 for 98.54€ - still needs 0.0W
provider01 sold 97.13000000000001W of excessive energy to house03 for 19.43€- ha

In [43]:
# Aggregator of Communities

class CommunityAggregator(PowerAggregator):
    
    def __init__(self, objs, max_iterations=500, transferCost=0.4):
        
        super().__init__(objs=objs, max_iterations=max_iterations)
        
        # For the transference of power between communities
        self.transferCost = transferCost
        
        # Location-aware buyers and sellers
        self.seller_location = []
        self.buyer_location = []
        
    def getSellers(self):
        self.sellers = []
        self.seller_location = []
        for i in np.arange(len(self.controlling)):
            for j in np.arange(len(self.controlling[i])):
                if self.controlling[i][j].canSell:
                    self.sellers.append(self.controlling[i][j])
                    self.seller_location.append(i)
        return
    
    def getBuyers(self):
        self.buyers = []
        self.buyer_location = []
        for i in np.arange(len(self.controlling)):
            for j in np.arange(len(self.controlling[i])):
                if self.controlling[i][j].isBuying:
                    self.buyers.append(self.controlling[i][j])
                    self.buyer_location.append(i)
        return
    
    def _step(self, step):
        # Do the initial iteration for all the controlled objects
        for obj in np.arange(len(self.controlling)):
            for sub_obj in np.arange(len(self.controlling[obj])):
                self.controlling[obj][sub_obj]._step(step)
            
        self.getSellers()
        self.getBuyers()
        return
    
    def energyTrading(self, step, buyer, sellers):
        # First: Check energy buyers
        # Second: Check the cheapest sellers (powerCost values)
        # Third: Update values move to next controlled object
        
        best_seller = self.cheapest_seller(buyer)
        
        sold_amount = best_seller.sellEnergy(step, buyer.toImport, buyer.identifier)
        bought_amount = buyer.buyEnergy(step, sold_amount, 
                                        best_seller.identifier, 
                                        best_seller.powerCost)
        
        self.getSellers()
        self.getBuyers()
        return
        
    def cheapest_seller(self, buyer):
        seller_costs = []
        for i in np.arange(len(self.sellers)):
            if self.seller_location[i] != self.buyer_location[0]:
                #print(self.sellers[i].identifier)
                #print(buyer.identifier)
                seller_costs.append(self.sellers[i].powerCost + self.transferCost)
            else:
                seller_costs.append(self.sellers[i].powerCost)
        #print([self.sellers[i].identifier for i in np.arange(len(self.sellers))])
        #print(seller_costs)
        return self.sellers[np.argmin(seller_costs)]
    
    

In [45]:
# Community

house_a1 = Household(identifier='house_a1', 
                     production_forecasts=np.round(agent01['yhat'], 2), 
                     consumption_forecasts=np.round(agent01['yhat_cons'], 2), 
                     production_gt=np.round(agent01['y'], 2), 
                     consumption_gt=np.round(agent01['y_cons'], 2),
                     powerCost=0.05)

house_a2 = Household(identifier='house_a2',
                     production_forecasts=np.round(agent05['yhat'], 2),
                     consumption_forecasts=np.round(agent05['yhat_cons'], 2),
                     production_gt=np.round(agent05['y'], 2),
                     consumption_gt=np.round(agent05['y_cons'], 2),
                     powerCost=0.07)

provider_a1 = Provider(identifier='provider_a1',
                       production_forecasts=[9999 for i in np.arange(agent01.shape[0])],
                       consumption_forecasts=[0 for i in np.arange(agent01.shape[0])],
                       production_gt=[9999 for i in np.arange(agent01.shape[0])],
                       consumption_gt=[0 for i in np.arange(agent01.shape[0])],
                       powerCost=0.40)

house_b1 = Household(identifier='house_b1', 
                     production_forecasts=np.round(agent01['yhat']*1.3, 2), 
                     consumption_forecasts=np.round(agent01['yhat_cons']*0.8, 2), 
                     production_gt=np.round(agent01['y']*1.3, 2), 
                     consumption_gt=np.round(agent01['y_cons']*0.8, 2),
                     powerCost=0.02)

house_b2 = Household(identifier='house_b2',
                     production_forecasts=np.round(agent05['yhat']*1.1, 2),
                     consumption_forecasts=np.round(agent05['yhat_cons']*1.2, 2),
                     production_gt=np.round(agent05['y']*1.1, 2),
                     consumption_gt=np.round(agent05['y_cons']*1.2, 2),
                     powerCost=0.07)

provider_b1 = Provider(identifier='provider_b1',
                       production_forecasts=[9999 for i in np.arange(agent01.shape[0])],
                       consumption_forecasts=[0 for i in np.arange(agent01.shape[0])],
                       production_gt=[9999 for i in np.arange(agent01.shape[0])],
                       consumption_gt=[0 for i in np.arange(agent01.shape[0])],
                       powerCost=0.60)

community_01 = CommunityAggregator(objs=[[house_a1, house_a2, provider_a1], 
                                         [house_b1, house_b2, provider_b1]],
                                   max_iterations=int(1440/5),
                                   transferCost=0.3)
community_01.iterate()

ITERATION 0
provider_a1 sold 610.35W of excessive energy to house_a1 for 244.14€- has 9388.65W remaining
house_a1 bought 610.35W of required energy from provider_a1 for 244.14€ - still needs 0.0W
provider_a1 sold 81.62W of excessive energy to house_a2 for 32.65€- has 9307.029999999999W remaining
house_a2 bought 81.62W of required energy from provider_a1 for 32.65€ - still needs 0.0W
provider_b1 sold 488.28W of excessive energy to house_b1 for 292.97€- has 9510.72W remaining
house_b1 bought 488.28W of required energy from provider_b1 for 292.97€ - still needs 0.0W
provider_b1 sold 97.92W of excessive energy to house_b2 for 58.75€- has 9412.8W remaining
house_b2 bought 97.92W of required energy from provider_b1 for 58.75€ - still needs 0.0W
ITERATION 1
provider_a1 sold 610.35W of excessive energy to house_a1 for 244.14€- has 9388.65W remaining
house_a1 bought 610.35W of required energy from provider_a1 for 244.14€ - still needs 0.0W
provider_a1 sold 76.62W of excessive energy to house_a2

provider_a1 sold 30.909999999999997W of excessive energy to house_a2 for 12.36€- has 9273.380000000001W remaining
house_a2 bought 30.909999999999997W of required energy from provider_a1 for 12.36€ - still needs 0.0W
provider_b1 sold 551.77W of excessive energy to house_b1 for 331.06€- has 9447.23W remaining
house_b1 bought 551.77W of required energy from provider_b1 for 331.06€ - still needs 0.0W
provider_b1 sold 42.13999999999999W of excessive energy to house_b2 for 25.28€- has 9405.09W remaining
house_b2 bought 42.13999999999999W of required energy from provider_b1 for 25.28€ - still needs 0.0W
ITERATION 220
provider_a1 sold 674.87W of excessive energy to house_a1 for 269.95€- has 9324.13W remaining
house_a1 bought 674.87W of required energy from provider_a1 for 269.95€ - still needs 0.0W
provider_a1 sold 44.56W of excessive energy to house_a2 for 17.82€- has 9279.57W remaining
house_a2 bought 44.56W of required energy from provider_a1 for 17.82€ - still needs 0.0W
provider_b1 sold 5